<a href="https://colab.research.google.com/github/darimakijou/exant-calc-merc-storia/blob/master/exant_calc_merc_storia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ユガラボユニットデータ（unitsdatajson）を用いたエグザント計算機

## **機能**
ユガラボ神（[yugalab.net](https://yugalab.net)）で作成したユニット所持情報を元にエグザント計算を実行し、期待値の高い順にリストアップするプログラムです。

所持ユニットのファイルは自前で用意する必要がある関係上、Google Colaboratory上でのプログラムとしました。

## **注意事項**
実行にはGoogleアカウントが必要です。

# **実行方法**
## **準備**
### 1. ユニット管理データ
ユガラボ神のサイト上で所持ユニットを登録してください。
データはエクスポート可能です（[ユガラボ神へのリンク](https://yugalab.net/mrst/add?mode=export)）。

コピー＆ペーストできようにしておきます。


In [ ]:
#@markdown # **ユニット管理データの貼り付け**
#@markdown 以下にコピー＆ペーストします。
userdata = '' #@param {type:"string"}

## **計算実行**
このページの一番上のメニューで、ランタイム＞すべてのセルを実行　をクリックします。

エラーがなければ計算／結果表示セルに結果が出力されます。


In [ ]:
#@markdown # **データのインポート**
#@markdown ## ユニットデータファイル(unitsdatajson)のパス:
#@markdown ※基本的に変更する必要はありません
unitsdata_path = "https://raw.githubusercontent.com/darimakijou/units-data-merc-storia/master/unitsdata.json" #@param {type:"string"}

!wget -N -q $unitsdata_path # ファイルをダウンロード
# !wget -N -q $userdata_path

#@markdown セルの実行後、
#@markdown > .  ..  .config	sample_data  unitsdata.json

#@markdown となっていればOK
!ls -a

In [ ]:
#@markdown ## 計算オプション
#@markdown 変更した場合は、「計算／結果表示セル」の▶ボタンを押す。
from ipywidgets import IntSlider, GridspecLayout, IntText, Text, FloatText, RadioButtons, Checkbox, FloatSlider

style = {'description_width': 'initial'}
grid_rune = GridspecLayout(2, 2)
grid_rune[0,0] = rune_attack = FloatText(value=33, description='アタック: ',disabled=False, style=style)
grid_rune[0,1] = rune_quick = FloatText(value=33, description='クイック: ',disabled=False, style=style)
grid_rune[1,0] = rune_exant = FloatText(value=33, description='エグザント: ',disabled=False, style=style)
grid_rune[1,1] = rune_fencer = FloatText(value=0,  description='フェンサー: ',disabled=False, style=style)

grid_facility = GridspecLayout(1, 2)
grid_facility[0,0] = training_grounds = FloatText(value=0, description='訓練所: ',disabled=False, style=style)
grid_facility[0,1] = weapon_facilitiy = FloatText(value=0, description='武器施設: ',disabled=False, style=style)

grid_countrymind = GridspecLayout(1, 2)
grid_countrymind[0,0] = country_mind = RadioButtons( options=[0, 5, 7], value=7, description='ルーン強化(%):', disabled=False, orientation='horizontal', style=style)
grid_countrymind[0,1] = assist_percent = FloatText(value=15.0, description='防護力(%): ',disabled=False, style=style)

grid_partybonus = GridspecLayout(1, 3)
grid_partybonus[0,0] = bonus_lead = RadioButtons( options=[0, 20, 30, 35], value=0, description='リードボーナス(%):', disabled=False, orientation='horizontal', style=style)
grid_partybonus[0,1] = bonus_rainbow = Checkbox( value=False, description='レインボーズ', disabled=False, indent=False , style=style)
bonus_rainbow = 5 if bonus_rainbow else 0
grid_partybonus[0,2] = bonus_varieties = Checkbox( value=False, description='バラエティーズ', disabled=False, indent=False , style=style)
bonus_varieties = 10 if bonus_varieties else 0

grid_others = GridspecLayout(1, 3)
grid_others[0,0] = time = FloatText(value=30.0, description='経過時間(秒): ',disabled=False, style=style)
grid_others[0,1] = boss_HP = FloatSlider(value=50,min=0,max=100,step=0.1,description='ボス残り体力（％）: ',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='.1f', style=style)
grid_others[0,2] = guts = IntText(value=30,  description='ガッツ: ',disabled=False, style=style)

grid_element = GridspecLayout(7, 2)
grid_element[0,0] = calc_element = RadioButtons( options=['考慮しない', '炎', '水','風','光','闇'], value='考慮しない', description='属性考慮: ', disabled=False, style=style )
grid_element[1,0] = cm_elem      = RadioButtons( options=[0,6,12,18], value=0, description='カントリーマインドの主効果: ', disabled=False, style=style)
if cm_elem.value:
  pass
grid_element[1,1] = cm_sub_elem = Checkbox( value=False, description='カントリーマインドの副効果', disabled=False, indent=False , style=style)
if cm_sub_elem.value:
  pass
grid_element[2,0] = rune_elem    = FloatText(value=33, description='属性: ',disabled=False, style=style)
grid_element[2,1] = rune_increase  = FloatText(value=33, description='イクリーズ: ',disabled=False, style=style)
grid_element[3,0] = rune_enhancer = FloatText(value=0, description='エンハンサー: ',disabled=False, style=style)
grid_element[3,1] = party_element = RadioButtons( options=[0,1,2,3,4,5], value=3, description='PTの有利属性ユニット: ', disabled=False, style=style)
grid_element[4,0] = elemental_facility = FloatText(value=0, description='神殿: ',disabled=False, style=style)
grid_element[5,0] = seed_break       = FloatText(value=15.0, description='ブレイク(%): ',disabled=False, style=style)
grid_element[5,1] = seed_break_count = IntText(value=3,  description='ブレイクを打った回数: ',disabled=False, style=style)
grid_element[6,0] = seed_increase     = FloatText(value=15.0, description='イクリーズ: ',disabled=False, style=style)

print("＜ルーン＞")
display(grid_rune)
print("-------------------------------------------------------------------------------------------------------------------------------------------")
print("＜施設＞")
display(grid_facility)
print("-------------------------------------------------------------------------------------------------------------------------------------------")
print("＜カントリーマインド＞")
display(grid_countrymind)
print("-------------------------------------------------------------------------------------------------------------------------------------------")
print("＜編成ボーナス＞")
display(grid_partybonus)
print("-------------------------------------------------------------------------------------------------------------------------------------------")
print("＜その他＞")
display(grid_others)
# print("-------------------------------------------------------------------------------------------------------------------------------------------")
# print("＜属性補正計算（エグザント計算には入らない）＞") # https://www.merrotten.net/entry/2017/02/24/180059#発動率上昇手段
# display(grid_element)

In [ ]:
#@markdown # ファイルからデータを読み込み
import pandas as pd
import json, os, pathlib, urllib.parse

# unitsdata, userdata = os.path.basename(unitsdata_path), os.path.basename(userdata_path) # ファイル名を取得
unitsdata = os.path.basename(unitsdata_path)# ファイル名を取得

# unitsdatajsonから計算に利用する要素を抽出してDataFrameを作成
unitsdataobj = pd.read_json(unitsdata,orient='records')
unitsdataobj_filtered = unitsdataobj.loc[['id', 'link', 'name', 'atkmax', 'spd', 'inter', 'reach', 'reacht', 'rarity', 'assault', 'combo', 'cfire', 'cwater', 'cwind', 'clight', 'cdark', 'style', 'typenum', 'weaponnum', 'growrate', 'grow']]
unitsdataobj= unitsdataobj_filtered.transpose()

# userdatajsonは読み込みに一工夫
# userdatapath = pathlib.Path(userdata)
# with open(userdatapath) as f:
#   json_open = f.read()
json_open = userdata
d = json.loads(json_open)
d_target = d['mrstunit_data']
s_target = json.dumps(d_target)

# ユーザーIDに文字列が付いているので数値のみid列を追加
userdataobj = pd.read_json(s_target).transpose()
userdataobj['id'] = userdataobj.index
userdataobj['id'] = userdataobj['id'].map(lambda x: int(x[9:]))

# unitsdataとuserdataをidを利用してマージ
unitsdata_merge = unitsdataobj.merge(userdataobj, left_on="id", right_on="id")

# NaNをすべて0に
unitsdata_merge = unitsdata_merge.fillna(0)

print("OK")

In [ ]:
#@markdown # unitsdatajsonから必要なデータを抽出
to_numeric_list = ['id', 'atkmax', 'spd', 'inter', 'reach', 'reacht', 'rarity', 'assault', 'combo', 'cfire', 'cwater', 'cwind', 'clight', 'cdark', 'style', 'typenum', 'weaponnum', 'growrate', 'have']
for key in to_numeric_list:
  unitsdata_merge[key] = pd.to_numeric(unitsdata_merge[key])
# growtypeから施設にかかる係数を定義
unitsdata_merge["facility_rate"] = unitsdata_merge['grow'].apply(lambda x: 1 if x == "fast" else 2/3 if x == "normal" else 0.5)
# 
unitsdata_merge["close_range"]  = unitsdata_merge["reacht"].apply(lambda x: 1 if x == 1 else 0)
unitsdata_merge["cm_factor"]   = unitsdata_merge["rarity"].apply(lambda x: 7 if x == 5 else 5)

#print(unitsdata_merge.dtypes) #型の確認

# データフレームのフィルタリング
unitsdata_merge = unitsdata_merge[(unitsdata_merge['weaponnum'] < 2) & (unitsdata_merge['style'] < 2)]
#print(unitsdata_merge)

print("OK")

In [ ]:
#@markdown # 出力される表のスタイル設定
# 出力される表のスタイル設定（一部動作しない）

# カーソルが乗ったセルをハイライトするスタイル設定
# https://qiita.com/code0327/items/2a6b51c11a6895d3e0b1
import IPython

def set_css_in_cell_output():
  display(IPython.display.HTML('''<style>
  table.dataframe td, table.dataframe th{
    border:1px solid #dadada;
  }
  table.dataframe th{
    font-family: 'Roboto','Noto',sans-serif;
    background: #eeeeee;
    font-weight: bold;
  }
  table.dataframe td{
    background: #ffffff;
  }
  table.dataframe tbody tr:hover th{
    color: #ffffff;
    background-color:#4169e1;
  }
  table.dataframe tr:hover td{
    background-color: #d7eeff;
  }
  </style>'''))

get_ipython().events.register('pre_run_cell', set_css_in_cell_output)

# 属性のセルを色分け（未動作）
def cell_style(value):
    if value == "炎":
        return "background-color: #F3AB93; color: #666666"
    elif value == "水":
        return "background-color: #88CCFB; color: #666666"
    elif value == "風":
        return "background-color: #B4CC99; color: #666666"
    elif value == "光":
        return "background-color: #E4D667; color: #666666"
    elif value == "闇":
        return "background-color: #C391E9; color: #666666"

# n体の確率の列をハイライト（未動作）
def highlight(data, color='lightskyblue'):
    return 'background-color: {}'.format(color)

print("OK")


In [ ]:
#@markdown ## 結果表示オプション
#@markdown 変更したら「計算／結果表示セル」の▶ボタンを押す
grid_displayoption = GridspecLayout(2, 3)
grid_displayoption[0,0] = calc_combo = RadioButtons( options=[1, 2, 3, 4, 5], value=2, description='計算対象の段数: ', disabled=False, orientation='horizontal', style=style)
grid_displayoption[0,1] = calc_assault = RadioButtons( options=["1", "2", "3", "4", "5", "まとめて"], value="5", description='ソート結果(n体の確率): ', disabled=False)
grid_displayoption[0,2] = max_rows   = IntText(value=30,  description='表示ユニット数: ',disabled=False, style=style)
grid_displayoption[1,0] = calc_have   = Checkbox( value=True, description='所持ユニットのみ表示', disabled=False, indent=False , style=style)

print("＜表示オプション＞")
display(grid_displayoption)


In [ ]:
#@markdown ## 計算／結果表示セル
unitsdata_applied = pd.DataFrame()
to_applied_dict = {"name":'ユニット名', "rarity":'レア', "typenum":"属性", "reach":"リーチ", "assault":"攻撃数", "combo":"段数", "weaponnum": "武器種", "have": "所持", "link": "link"} #  "atkmax":"atkmax", "inter":"inter",
for key, val in to_applied_dict.items():
  unitsdata_applied[val] = unitsdata_merge[key]

# 実際の攻撃間隔
unitsdata_merge["actual_interval"] = unitsdata_merge.apply(lambda x: x['inter']*(1-rune_quick.value/100*(1+min(country_mind.value, x["cm_factor"]) /100))*0.6 ,axis=1)
# 実際の攻撃力
unitsdata_merge["actual_attack"]  = unitsdata_merge.apply(lambda x: (x['atkmax']*(1+(rune_attack.value/100)*(1+min(country_mind.value, x["cm_factor"])/100)+rune_fencer.value*x['close_range']/500+ assist_percent.value/100+training_grounds.value/100+weapon_facilitiy.value*x['facility_rate']/100)+x['atkmax']*(1+(rune_attack.value/100)*(1+min(country_mind.value, x["cm_factor"])/100)+rune_fencer.value*x['close_range']/500)*((1+bonus_lead.value*x['close_range']/100)*(1+bonus_varieties/100)*(1+bonus_rainbow/100)-1)),axis=1)
# 属性攻撃力
# options=['考慮しない', '炎', '水','風','光','闇']
if calc_element.value == "考慮しない":
  pass
elif calc_element.value == "炎":
  unitsdata_merge["elemental_damage"] = unitsdata_merge.apply(lambda x:  (x["cfire"] * (1 + cm_elem.value) + cm_sub_elem.value) * (1 + rune_elem.value) * (1 + elemental_facility.value) + seed_break.value * seed_break_count.value + seed_increase.value + rune_increase.value / 3 + rune_enhancer.value / 15 * party_element.value,axis=1)
elif calc_element.value == "水":
  unitsdata_merge["elemental_damage"] = unitsdata_merge.apply(lambda x:  (x["cwater"] * (1 + cm_elem.value) + cm_sub_elem.value) * (1 + rune_elem.value) * (1 + elemental_facility.value) + seed_break.value * seed_break_count.value + seed_increase.value + rune_increase.value / 3 + rune_enhancer.value / 15 * party_element.value,axis=1)
elif calc_element.value == "風":
  unitsdata_merge["elemental_damage"] = unitsdata_merge.apply(lambda x:  (x["cwind"] * (1 + cm_elem.value) + cm_sub_elem.value) * (1 + rune_elem.value) * (1 + elemental_facility.value) + seed_break.value * seed_break_count.value + seed_increase.value + rune_increase.value / 3 + rune_enhancer.value / 15 * party_element.value,axis=1)
elif calc_element.value == "光":
  unitsdata_merge["elemental_damage"] = unitsdata_merge.apply(lambda x:  (x["clight"] * (1 + cm_elem.value) + cm_sub_elem.value) * (1 + rune_elem.value) * (1 + elemental_facility.value) + seed_break.value * seed_break_count.value + seed_increase.value + rune_increase.value / 3 + rune_enhancer.value / 15 * party_element.value,axis=1)
elif calc_element.value == "闇":
  unitsdata_merge["elemental_damage"] = unitsdata_merge.apply(lambda x:  (x["cdark"] * (1 + cm_elem.value) + cm_sub_elem.value) * (1 + rune_elem.value) * (1 + elemental_facility.value) + seed_break.value * seed_break_count.value + seed_increase.value + rune_increase.value / 3 + rune_enhancer.value / 15 * party_element.value,axis=1)
  
# 「１段の確率」やっしーさん計算表
unitsdata_merge["prob_per_combo"]  = unitsdata_merge.apply(lambda x: ((((((x['actual_attack']/x['combo'] )+((x['actual_attack']/x['combo'])*(guts.value-1)/10))*(time.value*0.2)**0.5)*(((1-(boss_HP.value/100))**4.22)))*((rune_exant.value)*(1+min(country_mind.value,x["cm_factor"]) /100))**2)/4080000000+0.005)*100,axis=1)
# 1体の確率
unitsdata_merge["prob_1"]  = unitsdata_merge.apply(lambda x: x['prob_per_combo'] * x['combo'] * min(x['assault'], 1) / x['actual_interval'],axis=1)
# 2体の確率
unitsdata_merge["prob_2"]  = unitsdata_merge.apply(lambda x: x['prob_per_combo'] * x['combo'] * min(x['assault'], 2) / x['actual_interval'],axis=1)
# 3体の確率
unitsdata_merge["prob_3"]  = unitsdata_merge.apply(lambda x: x['prob_per_combo'] * x['combo'] * min(x['assault'], 3) / x['actual_interval'],axis=1)
# 4体の確率
unitsdata_merge["prob_4"]  = unitsdata_merge.apply(lambda x: x['prob_per_combo'] * x['combo'] * min(x['assault'], 4) / x['actual_interval'],axis=1)
# 5体の確率
unitsdata_merge["prob_5"]  = unitsdata_merge.apply(lambda x: x['prob_per_combo'] * x['combo'] * min(x['assault'], 5) / x['actual_interval'],axis=1)

calc_col_dict = {"actual_interval": "攻撃間隔", "actual_attack":"攻撃力", "prob_per_combo": "１段の確率", "prob_1": "1体の確率", "prob_2": "2体の確率", "prob_3": "3体の確率", "prob_4": "4体の確率", "prob_5": "5体の確率"}
for key, val in calc_col_dict.items():
  unitsdata_applied[val] = unitsdata_merge[key]
unitsdata_applied = unitsdata_applied.reindex(columns=["ユニット名", "レア", "属性", "リーチ", "攻撃数", "段数", "武器種", "攻撃間隔", "攻撃力", "１段の確率","1体の確率","2体の確率","3体の確率","4体の確率","5体の確率","所持", "link" ])
unitsdata_applied["link"] = unitsdata_applied["link"].apply(lambda x: urllib.parse.unquote(x))
weapon_dict = {1: "斬撃", 2:"突撃", 3:"打撃", 4:"弓矢", 5:"魔法", 6:"銃弾", 7:"回復"}
unitsdata_applied["武器種"] = unitsdata_merge["weaponnum"].apply(lambda x: weapon_dict[x])
elemtype_dict = {1:"炎", 2:"水", 3:"風", 4:"光", 5:"闇"}
unitsdata_applied["属性"] = unitsdata_merge["typenum"].apply(lambda x: elemtype_dict[x])

# 動かない
unitsdata_applied.style.applymap(cell_style) #, axis=1)
#unitsdata_applied.style.applymap(highlight, subset=["n体の確率"])

# DataFrameの表示設定
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.precision',3)         # 表示桁数
pd.set_option('max_colwidth', 400)
pd.set_option('display.width', 400)
#pd.set_option('display.colheader_justify', 'left')

if calc_have:
  unitsdata_applied = unitsdata_applied[(unitsdata_merge['have'] > 0)]

if calc_have:
  unitsdata_applied = unitsdata_applied[(unitsdata_merge['combo'] >= calc_combo.value)]

if calc_assault.value == "まとめて":
  # 5体の確率
  unitsdata_applied["n体確率の最大値"]  = unitsdata_merge.apply(lambda x: max(x["prob_1"],x["prob_2"],x["prob_3"],x["prob_4"],x["prob_5"]),axis=1)
  unitsdata_applied = unitsdata_applied.drop(["1体の確率","2体の確率","3体の確率","4体の確率","5体の確率",], axis=1)
  unitsdata_applied = unitsdata_applied.sort_values("n体確率の最大値", ascending=False)
  unitsdata_applied = unitsdata_applied.reindex(columns=["ユニット名", "レア", "属性", "リーチ", "攻撃数", "段数", "武器種", "攻撃間隔", "攻撃力", "１段の確率","n体確率の最大値","所持", "link" ])
elif int(calc_assault.value) ==1:
  unitsdata_applied = unitsdata_applied.sort_values("1体の確率", ascending=False)
elif int(calc_assault.value) ==2:
  unitsdata_applied = unitsdata_applied.sort_values("2体の確率", ascending=False)
elif int(calc_assault.value) ==3:
  unitsdata_applied = unitsdata_applied.sort_values("3体の確率", ascending=False)
elif int(calc_assault.value) ==4:
  unitsdata_applied = unitsdata_applied.sort_values("4体の確率", ascending=False)
elif int(calc_assault.value) ==5:
  unitsdata_applied = unitsdata_applied.sort_values("5体の確率", ascending=False)

# 動かない
# https://www.it-swarm.dev/ja/pandas/pandas%EF%BC%86jupyter-notebook%E3%81%A7%E3%82%AF%E3%83%AA%E3%83%83%E3%82%AF%E5%8F%AF%E8%83%BD%E3%81%AA%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%AF%E3%82%92%E5%90%AB%E3%82%80%E3%83%86%E3%83%BC%E3%83%96%E3%83%AB%E3%82%92%E4%BD%9C%E6%88%90%E3%81%99%E3%82%8B%E6%96%B9%E6%B3%95/829990050/
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

unitsdata_applied.style.format({'link': make_clickable})

unitsdata_applied[:max_rows.value]


# 今後の機能追加
 - エグザント計算以外の計算

## 1. ユニットデータ（unitsdatajson）の保存方法
**ご注意：この方法でダウンロードできない場合、作者は対応できません。またユガラボ神に問い合わせるのもご遠慮ください（あくまでもわかる方むけの機能です）**

1. ユニットデータベースページでJavaScript Consoleを開きます。
2. Consoleにて以下のコマンドを入力します。実行するとダイアログがでるので保存します。

```
data=unitsdatajson

(function(console){

console.save = function(data, filename){

    if(!data) {
        console.error('Console.save: No data')
        return;
    }

    if(!filename) filename = 'console.json'

    if(typeof data === "object"){
        data = JSON.stringify(data, undefined, 4)
    }

    var blob = new Blob([data], {type: 'text/json'}),
        e    = document.createEvent('MouseEvents'),
        a    = document.createElement('a')

    a.download = filename
    a.href = window.URL.createObjectURL(blob)
    a.dataset.downloadurl =  ['text/json', a.download, a.href].join(':')
    e.initMouseEvent('click', true, false, window, 0, 0, 0, 0, 0, false, false, false, false, 0, null)
    a.dispatchEvent(e)
 }
})(console)

console.save(data,)
```

# ユガラボユニットデータ（unitsdatajson）を用いたエグザント計算機

## **1. 機能**
ユガラボ神（[yugalab.net](https://yugalab.net)）で作成したユニット所持情報を元にエグザント計算を実行し、期待値の高い順にリストアップするプログラムです。

所持ユニットのファイルは自前で用意する必要がある関係上、Google Colaboratory上でのプログラムとしました。

間違いなどございましたら、指摘いただけるとうれしいです。

## **2. ユニットデータについて**
ユガラボ神（[yugalab.net](https://yugalab.net)）のユニットデータはファイルで取得可能です。
利用可能とのことだったので、利用させていただくことにしました。

<blockquote class="twitter-tweet"><p lang="ja" dir="ltr">ちなみにユニットのデータを一括で取り込みたい場合、ユニットデータベースページでJavaScript Console開いて&quot;unitsdatajson&quot;で打ち込むとObject返ってくるんでそっからなんとかしてください(わかる方むけ)</p>&mdash; ユーリ (@yugalab) <a href="https://twitter.com/yugalab/status/1273273295983439873?ref_src=twsrc%5Etfw">June 17, 2020</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

## **3. エグザント計算について**
ヤッシー様（[@yaswiqa](https://twitter.com/yaswiqa)）の公開されたスプレッドシートの数式をそのまま利用しています。
1体確率が１位のまつかぜで数値を確認しています（ユガラボ神のユニットデータとは攻撃間隔の初期値が異なるため、一致しません）。

<blockquote class="twitter-tweet"><p lang="ja" dir="ltr">フィクセイト計算機は整いましたので配布します<br>何秒後にフィクセイトのDPS上昇効果が最大になるかを大まかですが計算するシートになります。<br>1枚目はエグザント、2枚目がフィクセイトです。間違い等あればご指摘下さい<a href="https://t.co/UVWmN3E3Ww">https://t.co/UVWmN3E3Ww</a></p>&mdash; ヤッシー家のメルストメモ帳 (@yaswiqa) <a href="https://twitter.com/yaswiqa/status/1290126593805844480?ref_src=twsrc%5Etfw">August 3, 2020</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>



# 参考文献
 - マジックコマンドに引数をつかう

 https://qiita.com/Hyperion13fleet/items/8951e9ea4ad06764bdad

 - jsonファイルからDataFrameを作成

  https://note.nkmk.me/python-pandas-read-json/
 
 - 行列入れ替え

 https://note.nkmk.me/python-pandas-t-transpose/

 

# 参考サイト
1. ユニットデータ／ユニット管理データ

 ユガラボ神（[https://yugalab.net](https://yugalab.net)）

2. 計算式

 ヤッシー様公開のスプレッドシート（[リンク](https://t.co/UVWmN3E3Ww)）

3. メルストの英単語参考

 Merc Storia English Wiki

 https://merc-storia-english.fandom.com/wiki/Merc_Storia_English_Wiki